In [7]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd
import xmltodict

json형태로 받아서 데이터프레임(df)만들기
+) 함수이름은 수업시간에 했던 자료와 동일한 이름으로 사용

In [8]:
# [code1] url 접속을 요청하고 응답을 받아서 반환
def getRequestUrl(url):
    req = urllib.request.Request(url)  #매개변수로 받은 url에 대한 요청을 보낼 객체를 생성
    try:
        response = urllib.request.urlopen(req)  #요청 객체를 보내서 받은 응답 데이터를 response 객체에 저장

        #response 객체에 저장된 코드를 확인. 코드가 200이면 요청을 정상 처리한 것이므로 성공.
        # 메시지와 현재 시간을 파이썬 셸 창에 출력하고 응답을 utf-8 형식으로 디코딩하여 반환.
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [12]:
# [code2] 관광지숙박 정보의 오픈 API를 사용하여 데이터 요청 url을 만들고 [CODE 1]의 getRequestUrl(url)을 호출해서 받은 응답 데이터를 반환
ServiceKey = "TEo5fURVg6O3ChvfXOmzkr0IXbTl0d4VkfIj3JVTz0ctJ%2BNS0IjPHxLXlijxDlubeXvzd3ZlGksTn%2FHhACp8gA%3D%3D"
def getTourismStatsItem(area, category):
    service_url = "http://apis.data.go.kr/6450000/TouristAccoService/getTouristAcco"

    parameters = "?ServiceKey=" + ServiceKey #인증키
    parameters += "&Area=" + area
    parameters += "&Category=" + category

    url = service_url + parameters
    print(url)   #액세스 거부 여부 확인용 출력

    retData = getRequestUrl(url) #[CODE 1]
    # print(json.loads(retData))

    #xml형식의 데이터를 Json파일로 바꿔서 리스트 형식으로 맞추기
    retData = xmltodict.parse(retData)
    retData = json.dumps(retData) #인코딩
    print(retData)

    if (retData == None):
        return print(retData)
    else:
        return json.loads(retData)

In [13]:
def getTourismStatsService(area, category):
    result = []
    jsonData = getTourismStatsItem(area, category)
    print(jsonData)
    if (jsonData['response']['header']['resultMsg'] == 'OK'):
        for i in range(len(jsonData['response']['body']['items']['item'])): #입력한 내용과 일치하는 숙박업소의 개수(len)만큼 리스트에 해당값을 넣는것을 반복
            area = jsonData['response']['body']['items']['item'][i]['AREA'] #지역코드
            sno = jsonData['response']['body']['items']['item'][i]['SNO'] #SNO는 숙박업체 고유번호
            category = jsonData['response']['body']['items']['item'][i]['CATEGORY'] #숙박업체종류
            address1 = jsonData['response']['body']['items']['item'][i]['ADDRESS1'] #숙박업체 주소
            address2 = jsonData['response']['body']['items']['item'][i]['ADDRESS2'] #숙박업체 상세주소
            tel = jsonData['response']['body']['items']['item'][i]['TEL'] #숙박업체 전화번호
            name = jsonData['response']['body']['items']['item'][i]['NAME'] #숙박업체 이름
            result.append([sno, area, category, name, address1, address2, tel]) #데이터프레임으로 바꿀 수 있게 리스트 형식으로 반환
    return result

In [14]:
print("<< 전라북도 관광지 숙박 위치 정보 >>")
area = input(
    '지역코드(01-고창군 02-군산시 03-김제시 04-남원시 05-무주군 06-부안군 07-순창군 08-완주군 09-익산시 10-임실군 11-장수군 12-전주시 13-진안군 14-진안군) : ')
category = input('숙박시설분류(01-호텔 02-굿스테이 03-리조트/콘도 04-모텔 05-팬션/민박 06-유스호스텔/수련원 07-한옥) : ')

#result에 getTourismStatsService 함수의 리스트값을 넣어줌
result = getTourismStatsService(area, category)  # [CODE 3]

# 컬럼 네임설정
columns = ["고유번호", "지역코드", '숙박종류', "이름", "주소", "상세주소", '전화번호']
result_df = pd.DataFrame(result, columns=columns)
result_df.to_csv(f'./{area}_{category}.csv', index=False, encoding='cp949')

<< 전라북도 관광지 숙박 위치 정보 >>
http://apis.data.go.kr/6450000/TouristAccoService/getTouristAcco?ServiceKey=TEo5fURVg6O3ChvfXOmzkr0IXbTl0d4VkfIj3JVTz0ctJ%2BNS0IjPHxLXlijxDlubeXvzd3ZlGksTn%2FHhACp8gA%3D%3D&Area=12&Category=01
[2022-12-07 14:37:57.909970] Url Request Success
{"response": {"header": {"resultCode": "200", "resultMsg": "OK"}, "body": {"totalCount": "5", "items": {"item": [{"IMG": "http://www.gojb.net/gwmsData/tourData/LODG_M/p1329724834083-1.jpg", "AREA": "12", "SNO": "1402", "CATEGORY": "01", "OVERVIEW": "\ub974\uc2dc\uc5d8 \ud638\ud154\uc740 \uace0\uac1d \uc5ec\ub7ec\ubd84\ub4e4\uc5d0\uac8c \uc2e0\ub8b0\uac10\uc744 \uc8fc\uae30 \uc704\ud558\uc5ec \uc804\ub77c\ubd81\ub3c4\uc640 \uc804\uc8fc\uc2dc\uc758 \uc9c0\uc815(\uc804\ub77c\ubd81\ub3c4 \uacf5\uace0 \uc81c2011-791\ud638)\uc744 \ubc1b\uc558\uc73c\uba70,\r\n\ud55c\uad6d\uad00\uad11\uacf5\uc0ac \uad7f\uc2a4\ud14c\uc774 \uc2b9\uc778(KTO-Cer-11-0023)\uc744 \ubc1b\uc558\uc2b5\ub2c8\ub2e4.", "TEL": "063-0245-4848", "ADDRESS1": "\uc804